### PIPELINE OVERVIEW
+ graph with genes as nodes and edges
+ generate a matrix from nodes based on specific parameters (features, random walk, shortest path..)
+ multidimensional matrices > embedding with tsne into 2D or 3D 
----------
+ Visualization typologies:
+ 2D PORTRAIT
+ 3D PORTRAIT
+ 3D LANDSCAPE
+ 3D SPHERE
----------

In [1]:
from multidimvis_main import *

_____
# SELECT ONE NETWORK GRAPH
_____

In [ ]:
################################################
#
# Y E A S T 
#
################################################

organism = 'Yeast'

data = pickle.load( open( "input/BIOGRID-ORGANISM-Saccharomyces_cerevisiae_S288c-3.5.185.mitab.pickle", "rb" ) )

Counter(data['Interaction Detection Method'])
Counter(data['Interaction Types'])

filter_score = data[
                    #(data['Interaction Types'] == 'psi-mi:"MI:0915"(physical association)') +
                    (data['Interaction Types'] == 'psi-mi:"MI:0407"(direct interaction)') 
                    #&
                    #(data['Taxid Interactor A'] == "taxid:559292") & 
                    #(data['Taxid Interactor B'] == "taxid:559292") 
]

g = nx.from_pandas_edgelist(filter_score, '#ID Interactor A', 'ID Interactor B')
g.remove_edges_from(nx.selfloop_edges(g)) #remove self loop

G_cere = g.subgraph(max(nx.connected_components(g), key=len)) # largest connected component (lcc)

# ESSENTIAL GENES 
cere_gene =pd.read_csv("input/Saccharomyces cerevisiae.csv",
           delimiter= ',',
           skipinitialspace=True)
essential_cere = cere_gene[(cere_gene['essentiality status'] == 'E')]
essential_genes_cere_list =  essential_cere['symbols'].tolist()

degree= dict(G_cere.degree())

mg = mygene.MyGeneInfo()
a = mg.querymany(essential_genes_cere_list, scopes='symbol', species=559292)
essential_genes_cere_names = pd.DataFrame.from_dict(a)
essential_genes_cere_entrez =  essential_genes_cere_names['entrezgene'].tolist()

cleaned_entrez_list = [x for x in essential_genes_cere_entrez if str(x) != 'nan']

degree_formatted={}
for k, v in degree.items():
    degree_formatted[k.replace("entrez gene/locuslink:","")] = v
    
index= []
essential = []
for i in cleaned_entrez_list:
    for (key, val) in degree_formatted.items():
        if i==key:
            index.append(key)
            essential.append(val)  

no_essential_cere = cere_gene[(cere_gene['essentiality status'] == 'NE')]
no_essential_genes_cere_list =  no_essential_cere['symbols'].tolist()
b = mg.querymany(no_essential_genes_cere_list, scopes='symbol', species=559292)
no_essential_genes_cere_names = pd.DataFrame.from_dict(b)
no_essential_genes_cere_entrez =  no_essential_genes_cere_names['entrezgene'].tolist()
cleaned_entrez_list_no = [x for x in no_essential_genes_cere_entrez if str(x) != 'nan']

index= []
no_essential = []
for i in cleaned_entrez_list_no:
    for (key, val) in degree_formatted.items():
        if i==key:
            index.append(key)
            no_essential.append(val)
            
#df_cere = pd.DataFrame({'essential': pd.Series(essential), 'no_essential': pd.Series(no_essential)})


no_ess_id = no_essential_genes_cere_names['entrezgene']
ess_id = essential_genes_cere_names['entrezgene']

G = G_cere

#edge_list = nx.write_edgelist(G, "Yeast_edgelist_directinteractiononly.txt")

In [2]:
################################################
#
# H U M A N 
#
################################################

organism = 'Human'

G = nx.read_edgelist('input/ppi_elist.txt',data=False)
# d_ent_sym, d_sym_ent = genent2sym()

d_gene_do = pickle.load( open( "input/d_gene_do.pkl", "rb" ) )
d_do_genes = pickle.load( open( "input/d_do_genes.pkl", "rb" ) )
d_do_names = pickle.load( open( "input/DO_names.pkl", "rb" ) )
d_names_do = {y:x for x,y in d_do_names.items()}

### GENE ID to SYN (e.g. HOVERINFO for nodes)

In [3]:
df_gene_sym = pd.read_csv('output_csv/DF_gene_symbol_Human.csv', index_col=0)
l_features = list((df_gene_sym.to_dict()).values())

In [4]:
print('Number of nodes i.e. genes: %s' %len(list(G.nodes())))
print('Number of edges: %s'%len(list(G.edges())))
print('Network density: %.1f%%' %(200.*len(list(G.edges()))/(len(list(G.nodes()))*len(list(G.nodes()))-1)))

Number of nodes i.e. genes: 16376
Number of edges: 309355
Network density: 0.2%


_____
# SELECT ONF OF THE FOLLOWING DISTANCE MATRICES
_____

### + STRUCTURAL : Graph structure based
+ Adjacency
+ Random Walk with Restart
+ Centrality measures

#### IMPORT PRECALCULATED 

In [5]:
DM_adj = pd.read_csv('output_csv/Adjacency_Dataframe_'+organism+'.csv', index_col=0)
DM_adj_r = DM_adj.round(4)

In [6]:
DM_adj = DM_adj_r

In [7]:
DM_adj

,66008,8473,2561,3759,22906,4928,1994,8481,81610,51361,...,10838,8001,51351,1551,51458,143903,10861,51471,221044,29965
66008,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
8473,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2561,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3759,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22906,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143903,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10861,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51471,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
221044,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
DM_m = pd.read_csv('output_csv/RWR_Dataframe_'+organism+'.csv', index_col=0)
DM_m_r = DM_m.round(4)

In [9]:
DM_m = DM_m_r

In [10]:
DM_m

,66008,8473,2561,3759,22906,4928,1994,8481,81610,51361,...,10838,8001,51351,1551,51458,143903,10861,51471,221044,29965
66008,0.0000,0.9928,0.9659,0.9801,0.9819,0.9933,0.9943,0.9936,0.9867,0.9880,...,1.0,1.0,1.0000,1.0,1.0,1.0000,1.0000,1.0,1.0,1.0
8473,0.9928,0.0000,0.9998,0.9999,0.9866,0.9999,0.9998,0.9999,0.9999,0.9999,...,1.0,1.0,1.0000,1.0,1.0,1.0000,1.0000,1.0,1.0,1.0
2561,0.9659,0.9998,0.0000,0.9993,0.9994,0.9996,0.9998,0.9998,0.9996,0.9996,...,1.0,1.0,0.9998,1.0,1.0,0.9998,0.9998,1.0,1.0,1.0
3759,0.9801,0.9999,0.9993,0.0000,0.9997,0.9998,0.9998,0.9999,0.9998,0.9998,...,1.0,1.0,1.0000,1.0,1.0,1.0000,1.0000,1.0,1.0,1.0
22906,0.9819,0.9866,0.9994,0.9997,0.0000,0.9871,0.9999,0.9999,0.9998,0.9997,...,1.0,1.0,1.0000,1.0,1.0,1.0000,1.0000,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143903,1.0000,1.0000,0.9998,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,...,1.0,1.0,0.9604,1.0,1.0,0.0000,0.9604,1.0,1.0,1.0
10861,1.0000,1.0000,0.9998,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,...,1.0,1.0,0.9604,1.0,1.0,0.9604,0.0000,1.0,1.0,1.0
51471,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.9997,1.0000,1.0000,1.0000,...,1.0,1.0,1.0000,1.0,1.0,1.0000,1.0000,0.0,1.0,1.0
221044,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,...,1.0,1.0,1.0000,1.0,1.0,1.0000,1.0000,1.0,0.0,1.0


In [6]:
df_centralities = pd.read_csv('output_csv/Features_centralities_Dataframe_'+organism+'.csv', index_col=0)

d_deghubs = dict(zip(G.nodes(),df_centralities['degs']))
d_clos = dict(zip(G.nodes(), df_centralities['clos']))
d_betw = dict(zip(G.nodes(), df_centralities['betw']))
d_eigen = dict(zip(G.nodes(), df_centralities['eigen']))

d_centralities = dict(zip(list(G.nodes),zip(d_deghubs.values(),d_clos.values(),d_betw.values(),d_eigen.values())))

#l_features = []
#for i in d_centralities.items():
#    k=list(i)
#    l_features.append(k)

DM_centralities = pd.DataFrame(distance.squareform(distance.pdist(df_centralities, 'cosine')))

DM_centralities.index = G.nodes()
DM_centralities.columns = G.nodes()

In [7]:
DM_centralities

,66008,8473,2561,3759,22906,4928,1994,8481,81610,51361,...,10838,8001,51351,1551,51458,143903,10861,51471,221044,29965
66008,0.000000,0.006356,0.000692,0.000369,0.000300,0.016879,0.463290,0.033598,0.000073,0.000013,...,1.060159e-03,1.056945e-03,1.059203e-03,1.046465e-03,1.058869e-03,1.059203e-03,1.059203e-03,1.064640e-03,1.063883e-03,1.003989e-03
8473,0.006356,0.000000,0.011225,0.009781,0.009364,0.002550,0.372517,0.010935,0.007739,0.006927,...,1.257835e-02,1.257018e-02,1.257788e-02,1.253433e-02,1.257674e-02,1.257788e-02,1.257788e-02,1.259095e-02,1.259384e-02,1.238522e-02
2561,0.000692,0.011225,0.000000,0.000051,0.000089,0.024363,0.494912,0.043815,0.000326,0.000522,...,3.962782e-05,3.875105e-05,3.918726e-05,3.675262e-05,3.912243e-05,3.918726e-05,3.918726e-05,4.081319e-05,4.009742e-05,2.929252e-05
3759,0.000369,0.009781,0.000051,0.000000,0.000013,0.022212,0.486267,0.040918,0.000124,0.000248,...,1.800162e-04,1.781710e-04,1.791053e-04,1.738533e-04,1.789667e-04,1.791053e-04,1.791053e-04,1.823835e-04,1.810461e-04,1.571258e-04
22906,0.000300,0.009364,0.000089,0.000013,0.000000,0.021607,0.484017,0.040154,0.000078,0.000198,...,2.395506e-04,2.390978e-04,2.401441e-04,2.342591e-04,2.399888e-04,2.401441e-04,2.401441e-04,2.408119e-04,2.423164e-04,2.139484e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143903,0.001059,0.012578,0.000039,0.000179,0.000240,0.026336,0.502554,0.046443,0.000588,0.000847,...,7.036998e-08,1.227396e-09,0.000000e+00,3.925712e-08,2.697809e-11,0.000000e+00,0.000000e+00,2.542837e-07,5.252571e-09,7.565385e-07
10861,0.001059,0.012578,0.000039,0.000179,0.000240,0.026336,0.502554,0.046443,0.000588,0.000847,...,7.036998e-08,1.227396e-09,0.000000e+00,3.925712e-08,2.697809e-11,0.000000e+00,0.000000e+00,2.542837e-07,5.252571e-09,7.565385e-07
51471,0.001065,0.012591,0.000041,0.000182,0.000241,0.026359,0.502681,0.046483,0.000590,0.000853,...,5.865623e-08,2.657031e-07,2.542837e-07,3.511814e-07,2.558217e-07,2.542837e-07,2.542837e-07,0.000000e+00,2.384522e-07,1.019050e-06
221044,0.001064,0.012594,0.000040,0.000181,0.000242,0.026360,0.502642,0.046474,0.000592,0.000852,...,6.853820e-08,1.155815e-08,5.252571e-09,7.322908e-08,6.032424e-09,5.252571e-09,5.252571e-09,2.384522e-07,0.000000e+00,8.828435e-07


### + FUNCTIONAL : Molecular Function

#### IMPORT PRECALCULATED 

In [13]:
DM_MF = pd.read_csv('output_csv/DistanceMatrix_goMF_Dataframe_Human_cosine.csv', index_col=0)

DM_MF_mod = DM_MF.round(decimals=4)

In [14]:
DM_MF = DM_MF_mod

In [15]:
DM_MF

,2,9,10,12,13,14,15,16,18,19,...,102724159,102724334,102724428,102724862,105371242,107987462,107987491,109703458,110117499,110599564
2,0.0000,1.0000,0.7868,0.6518,1.0,1.0,0.7868,1.0,1.0,0.8579,...,1.0,1.0000,1.0,0.7868,1.0,0.8492,0.8492,1.0,1.0,1.0
9,1.0000,0.0000,0.2929,1.0000,1.0,1.0,1.0000,1.0,1.0,1.0000,...,1.0,1.0000,1.0,1.0000,1.0,1.0000,1.0000,1.0,1.0,1.0
10,0.7868,0.2929,0.0000,0.5918,1.0,1.0,0.5000,1.0,1.0,0.8333,...,1.0,1.0000,1.0,1.0000,1.0,0.6464,0.6464,1.0,1.0,1.0
12,0.6518,1.0000,0.5918,0.0000,1.0,1.0,0.5918,1.0,1.0,0.8639,...,1.0,0.4226,1.0,1.0000,1.0,0.7113,0.7113,1.0,1.0,1.0
13,1.0000,1.0000,1.0000,1.0000,0.0,1.0,1.0000,1.0,1.0,1.0000,...,1.0,1.0000,1.0,1.0000,1.0,1.0000,1.0000,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107987462,0.8492,1.0000,0.6464,0.7113,1.0,1.0,0.6464,1.0,1.0,0.8821,...,1.0,1.0000,1.0,1.0000,1.0,0.0000,0.7500,1.0,1.0,1.0
107987491,0.8492,1.0000,0.6464,0.7113,1.0,1.0,0.6464,1.0,1.0,0.8821,...,1.0,1.0000,1.0,1.0000,1.0,0.7500,0.0000,1.0,1.0,1.0
109703458,1.0000,1.0000,1.0000,1.0000,1.0,1.0,1.0000,1.0,1.0,1.0000,...,1.0,1.0000,1.0,1.0000,1.0,1.0000,1.0000,0.0,1.0,1.0
110117499,1.0000,1.0000,1.0000,1.0000,1.0,1.0,1.0000,1.0,1.0,1.0000,...,1.0,1.0000,1.0,1.0000,1.0,1.0000,1.0000,1.0,0.0,1.0


### + FUNCTIONAL : Cellular Components

#### IMPORT PRECALCULATED 

In [ ]:
DM_CC = pd.read_csv('output_csv/DistanceMatrix_goCC_Dataframe_Human_cosine.csv', index_col=0)

DM_CC_mod = DM_CC.round(decimals=4)

In [ ]:
DM_CC = DM_CC_mod

In [ ]:
DM_CC

### + FUNCTIONAL : Biological Processes

#### IMPORT PRECALCULATED 

In [19]:
DM_BP = pd.read_csv('output_csv/DistanceMatrix_goBP_Dataframe_Human_cosine.csv', index_col=0)
    
DM_BP_mod = DM_BP.round(decimals=4)

In [20]:
DM_BP_mod = DM_BP.round(decimals=4)

In [21]:
DM_BP = DM_BP_mod

In [22]:
DM_BP

,1,2,9,10,12,13,14,15,16,18,...,102724862,105372315,105373297,105375355,105377641,107983988,107987462,107987491,109703458,110117499
1,0.0000,0.6838,1.0,1.0,0.4226,1.0000,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.6464,1.0,1.0,1.0
2,0.6838,0.0000,1.0,1.0,0.8174,1.0000,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0000,1.0,1.0,1.0
9,1.0000,1.0000,0.0,0.0,1.0000,0.2929,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0000,1.0,1.0,1.0
10,1.0000,1.0000,0.0,0.0,1.0000,0.2929,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0000,1.0,1.0,1.0
12,0.4226,0.8174,1.0,1.0,0.0000,1.0000,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.7959,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107983988,1.0000,1.0000,1.0,1.0,1.0000,1.0000,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0000,1.0,1.0,1.0
107987462,0.6464,1.0000,1.0,1.0,0.7959,1.0000,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0000,1.0,1.0,1.0
107987491,1.0000,1.0000,1.0,1.0,1.0000,1.0000,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0000,0.0,1.0,1.0
109703458,1.0000,1.0000,1.0,1.0,1.0000,1.0000,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0000,1.0,0.0,1.0


### + FUNCTIONAL : Disease Annotation

#### IMPORT PRECALCULATED 

In [23]:
DM_Disease = pd.read_csv('output_csv/DistanceMatrix_Disease_Dataframe_Human_cosine.csv', index_col=0)
    
DM_Disease_mod = DM_Disease.round(decimals=4)

In [24]:
DM_Disease = DM_Disease_mod

In [25]:
DM_Disease

,389289,4524,4353,27127,5925,10735,26259,1029,9649,1630,...,65125,6787,90678,10661,56479,63910,7287,11173,9962,5973
389289,0.0000,0.8360,0.7474,0.0000,0.7809,0.4380,0.0000,0.8075,0.0000,0.6349,...,0.9167,0.8174,0.9512,0.8946,0.8557,0.8333,0.9519,0.8557,0.8457,0.9385
4524,0.8360,0.0000,0.5027,0.8360,0.7065,0.8003,0.8360,0.7422,0.8360,0.8103,...,0.8086,0.8503,0.9680,0.8963,0.8579,0.8633,0.8343,0.8343,0.8988,0.9596
4353,0.7474,0.5027,0.0000,0.7474,0.7509,0.8580,0.7474,0.7893,0.7474,0.8309,...,0.7474,0.8155,0.9630,0.9201,0.8906,0.7895,0.8906,0.8177,0.8441,0.9378
27127,0.0000,0.8360,0.7474,0.0000,0.7809,0.4380,0.0000,0.8075,0.0000,0.6349,...,0.9167,0.8174,0.9512,0.8946,0.8557,0.8333,0.9519,0.8557,0.8457,0.9385
5925,0.7809,0.7065,0.7509,0.7809,0.0000,0.7127,0.7809,0.5784,0.7809,0.8000,...,0.9087,0.9200,0.9679,0.9307,0.8103,0.8174,0.8630,0.8735,0.9324,0.9595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63910,0.8333,0.8633,0.7895,0.8333,0.8174,0.9063,0.8333,0.8717,0.8333,0.9391,...,0.8333,0.8174,0.9024,0.7892,0.7113,0.0000,0.9038,0.7113,0.6914,0.8769
7287,0.9519,0.8343,0.8906,0.9519,0.8630,0.9730,0.9519,0.9074,0.9519,0.9649,...,0.8797,0.8946,0.9296,0.9391,0.6667,0.9038,0.0000,0.8750,0.9109,0.9645
11173,0.8557,0.8343,0.8177,0.8557,0.8735,0.9189,0.8557,0.9444,0.8557,0.9473,...,0.6392,0.8419,0.9155,0.8174,0.7500,0.7113,0.8750,0.0000,0.7327,0.8401
9962,0.8457,0.8988,0.8441,0.8457,0.9324,0.9133,0.8457,0.9109,0.8457,0.9437,...,0.8457,0.8310,0.9096,0.8048,0.7327,0.6914,0.9109,0.7327,0.0000,0.8860


_____
### SELECT ONE DISTANCE MATRIX

In [ ]:
#---------------
# STRUCTURAL
#---------------
#feature = 'adj'
#DM = DM_adj

#feature = 'rwr'
#DM = DM_m

#feature = 'centralities'
#DM = DM_centralities


#---------------
# FUNCTIONAL 
#---------------

#feature = 'bioproc'
#DM = DM_BP
#feat_matrix = pd.read_csv('output_csv/Features_GO_MolFunc_Dataframe_Human.csv', index_col=0)

#feature = 'molfunc'
#DM = DM_MF
#feat_matrix = pd.read_csv('output_csv/Features_GO_BiolProc_Dataframe_Human.csv', index_col=0)

feature = 'cellcomp'
DM = DM_CC
feat_matrix = pd.read_csv('output_csv/Features_GO_CellComp_Dataframe_Human.csv', index_col=0)


#feature = 'disease'
#DM = DM_Disease
#feat_matrix = pd.read_csv('output_csv/Features_Disease_Dataframe_Human.csv', index_col=0)


# set gene list (= G.nodes())
genes = []
for i in DM.index:
    genes.append(str(i))
    
genes_rest = [] 
for g in G.nodes():
    if g not in genes:
        genes_rest.append(g)

____________
# VISUAL PRE-SETTINGS
____________

In [ ]:
edge_width = 0.1
edge_color = 'lightgrey'

edge_col = 'lightgrey'
edge_colordark = 'dimgrey'
opacity_edges = 0.1

opacity_nodes = 1.0
node_edge_col = None

### NODE SIZE

In [ ]:
scalef= 0.2
size = list(draw_node_degree(G, scalef).values())

scalef= 0.025
size3d = list(draw_node_degree_3D(G, scalef).values())

### NODE + EDGE COLOUR PARAMETERS

+ COLOR > GRADIENT : BY DICT INPUT (e.g. different values for nodes, like Closeness Centrality) 

In [13]:
# ----------------------------------------------------        
# Define colour parameter with dict (gradient)
# ----------------------------------------------------        
color_method = 'clos'
d_to_be_coloured = d_clos # dict sorted by dict.values (that way the biggest value matches darkest colour of palette)


# Colouring
colour_groups = set(d_to_be_coloured.values())
colour_count = len(colour_groups)
pal = sns.color_palette('Reds', colour_count)
palette = pal.as_hex()

d_colourgroups = {}
for n in colour_groups:
    d_colourgroups[n] = [k for k in d_to_be_coloured.keys() if d_to_be_coloured[k] == n]
    
d_colourgroups_sorted = {key:d_colourgroups[key] for key in sorted(d_colourgroups.keys())}

d_val_col = {}
for idx,val in enumerate(d_colourgroups_sorted):
    for ix,v in enumerate(palette):
        if idx == ix:
            d_val_col[val] = v
d_node_colour = {}
for y in d_to_be_coloured.items(): # y[0] = node id, y[1] = val
    for x in d_val_col.items(): # x[0] = val, x[1] = (col,col,col)
        if x[0] == y[1]:
            d_node_colour[y[0]]=x[1]
            
# SORT dict based on G.nodes
d_node_colour_sorted = dict([(key, d_node_colour[key]) for key in G.nodes()])

l_col_clos = list(d_node_colour_sorted.values())

colours_hex = l_col_clos 


d_val_col = {}
for idx,val in enumerate(d_colourgroups_sorted):
    for ix,v in enumerate(pal):
        if idx == ix:
            d_val_col[val] = v
d_node_colour = {}
for y in d_to_be_coloured.items(): # y[0] = node id, y[1] = val
    for x in d_val_col.items(): # x[0] = val, x[1] = (col,col,col)
        if x[0] == y[1]:
            d_node_colour[y[0]]=x[1]
            
# SORT dict based on G.nodes
d_node_colour_sorted = dict([(key, d_node_colour[key]) for key in G.nodes()])

l_col = list(d_node_colour_sorted.values())

colours = l_col

____________
# 2D 
____________

In [29]:
# make classes + functions 
# for each layout typology

___
## PORTRAITS
___

### tSNE EMBEDDING 2D

In [ ]:
prplxty = 50 # range: 5-50 / default = 30
density = 1.0  # default 12.
l_rate = 1000 #default 200.
steps = 250 # min 250

metric = 'cosine'

In [ ]:
%%time 
tsne = embed_tsne_2D(DM, prplxty, density, l_rate, steps, metric)

In [ ]:
# MAIN (if not all G.nodes)
posG_tsne = get_posG_2D(genes, tsne)

# REST (if main = G.nodes then rest will be ignored / empty)
# generate circle coordinates for rest genes (without e.g. GO term or Disease Annotation)
t = np.random.uniform(0,2*np.pi,len(genes_rest))

# get center of points and define radius
xx=[]
yy=[]

for i in posG_tsne.values():
    xx.append(i[0])
    yy.append(i[1])

cx = sum(xx)/len(genes)
cy = sum(yy)/len(genes)

xm, ym = max(posG_tsne.values())
r = math.sqrt((cx - xm)**2 + (cy - ym)**2)

x = r*np.cos(t)
y = r*np.sin(t)

rest = []
for i,j in zip(x,y):
        rest.append((i,j))
        
posG_rest = dict(zip(genes_rest, rest))

posG_all = {**posG_tsne, **posG_rest}
posG_complete_tsne = {key:posG_all[key] for key in G.nodes()}

### tSNE CLUSTER Colors
+ https://jakevdp.github.io/PythonDataScienceHandbook/05.11-k-means.html

In [ ]:
# SPECTRAL from sklearn
from sklearn.cluster import SpectralClustering

df_posG = pd.DataFrame(posG_tsne).T

n_clus = 100
color_method = 'spectral'

model = SpectralClustering(n_clusters=n_clus,n_components = 2, affinity='nearest_neighbors')
clusterid = model.fit_predict(df_posG)

d_node_clusterid = dict(zip(genes, clusterid))

# colour parameter from cluster 
colours_unsort = color_nodes_from_dict(G, d_node_clusterid)

genes_val = [1]*len(genes_rest)
d_genes_rest = dict(zip(genes_rest, genes_val))
colours_rest = color_nodes_from_dict_same(G, d_genes_rest, 'dimgrey')

colours_all = {**colours_rest, **colours_unsort}

colours_sorted = {key:colours_all[key] for key in G.nodes}
colours = list(colours_sorted.values())

In [ ]:
plt.figure(figsize=(25,25))
plt.title(feature+' | t-SNE | Metric: '+metric, size= 16)

nx.draw_networkx_nodes(G, posG_complete_tsne, edgecolors = node_edge_col, linewidths = 0.25, node_color = colours, node_size=size, alpha = opacity_nodes)
#nx.draw_networkx_edges(G, posG_complete_tsne, width = edge_width, edge_color = edge_col, alpha = opacity_edges)

plt.box(False)

plt.savefig('output_plots/2Dtsne_'+feature+'_'+metric+'_'+str(prplxty)+'_'+str(density)+'_'+str(l_rate)+'_'+str(steps)
        +'_'+organism+'_'+color_method+'_nx.png')

plt.show()

### UMAP EMBEDDING 2D 
+https://umap-learn.readthedocs.io/en/latest/embedding_space.html

In [ ]:
n_neighbors = 4 # balance between local and global structure in the data
spread = 0.9
min_dist = 0.1 # defines how dense points are stacked together 
metric ='cosine'

In [ ]:
%%time 
umap = embed_umap_2D(DM, n_neighbors, spread, min_dist, metric)

In [ ]:
# MAIN (if not all G.nodes)
posG_umap = get_posG_2D(genes, umap)

# REST (if main = G.nodes then rest will be ignored / empty)
# generate circle coordinates for rest genes (without e.g. GO term or Disease Annotation)
t = np.random.uniform(0,2*np.pi,len(genes_rest))

# get center of points and define radius
xx=[]
yy=[]
for i in posG_umap.values():
    xx.append(i[0])
    yy.append(i[1])

cx = sum(xx)/len(genes)
cy = sum(yy)/len(genes)

xm, ym = max(posG_umap.values())
r = math.sqrt((xm-cx)**2 + (ym-cy)**2)

x = r*np.cos(t)
y = r*np.sin(t)

rest = []
for i,j in zip(x,y):
        rest.append((i,j))
        
posG_rest = dict(zip(genes_rest, rest))

posG_all = {**posG_umap, **posG_rest}
posG_complete_umap = {key:posG_all[key] for key in G.nodes()}

### UMAP CLUSTER Colors

In [ ]:
# SPECTRAL from sklearn
from sklearn.cluster import SpectralClustering

df_posG = pd.DataFrame(posG_umap).T

n_clus = 50
color_method = 'spectral'

model = SpectralClustering(n_clusters=n_clus,n_components = 2, affinity='nearest_neighbors')
clusterid = model.fit_predict(df_posG)

d_node_clusterid = dict(zip(genes, clusterid))

# colour parameter from cluster 
colours_unsort = color_nodes_from_dict(G, d_node_clusterid)

genes_val = [1]*len(genes_rest)
d_genes_rest = dict(zip(genes_rest, genes_val))
colours_rest = color_nodes_from_dict_same(G, d_genes_rest, 'dimgrey')

colours_all = {**colours_rest, **colours_unsort}

colours_sorted = {key:colours_all[key] for key in G.nodes}
colours = list(colours_sorted.values())

In [ ]:
plt.figure(figsize=(25,25))
plt.title(feature+' | UMAP | Metric: '+metric, size=16)

nx.draw_networkx_nodes(G, posG_complete_umap, edgecolors = node_edge_col, linewidths = 0.5, node_color=colours, node_size=size, alpha = opacity_nodes)
#nx.draw_networkx_edges(G, posG_complete_umap, width = edge_width, edge_color = edge_color, alpha = opacity_edges)
plt.box(False)

plt.savefig('output_plots/2Dumap_'+feature+'_'+metric+'_'+color_method+'_'+str(n_neighbors)+'_'+str(spread)+'_'+str(min_dist)+'_'+organism+'.png')

plt.show()

____________
# 3D
____________


____
## 3D PORTRAITS
____

In [ ]:
prplxty3d = 30 # range: 5-50 / default = 30.#
density3d = 1.2 # default 12.
l_rate3d = 1000 # default 200.
steps3d = 250 #min 250

metric = 'cosine'

In [ ]:
%%time
tsne_3D = embed_tsne_3D(DM, prplxty3d, density3d, l_rate3d, steps3d)

In [ ]:
# MAIN (if not all G.nodes)
posG_3Dtsne = get_posG_3D(genes, tsne_3D)

# REST (if genes = G.nodes then rest will be ignored / empty)

# get center of data points
xx=[]
yy=[]
zz=[]

for i in posG_3Dtsne.values():
    xx.append(i[0])
    yy.append(i[1])
    zz.append(i[2]) 

cx = sum(xx)/len(genes)
cy = sum(yy)/len(genes)
cz = sum(zz)/len(genes)

# generate spherical coordinates for rest genes (without e.g. GO term or Disease Annotation)
indices = arange(0, len(genes_rest))
phi = arccos(1 - 2*indices/len(genes_rest))
theta = pi * (1 + 5**0.5) * indices

xm, ym, zm = max(posG_3Dtsne.values())
r = math.sqrt((cx - xm)**2 + (cy - ym)**2 + (cz - zm)**2)
#r=1
x, y, z = cx+r*cos(theta) * sin(phi), cy+r*sin(theta) * sin(phi), cz+r*cos(phi)

rest_points = []
for i,j,k in zip(x,y,z):
    rest_points.append((i,j,k))

posG_rest = dict(zip(genes_rest, rest_points))

posG_all = {**posG_3Dtsne, **posG_rest}
posG_3D_complete_tsne = {key:posG_all[key] for key in G.nodes()}

### tSNE CLUSTER Colors

In [ ]:
# SPECTRAL from sklearn
from sklearn.cluster import SpectralClustering

df_posG = pd.DataFrame(posG_3Dtsne).T

n_clus = 50
color_method = 'spectral'

model = SpectralClustering(n_clusters=n_clus,n_components = 3, affinity='nearest_neighbors')
clusterid = model.fit_predict(df_posG)

d_node_clusterid = dict(zip(genes, clusterid))

# colour parameter from cluster 
colours_unsort = color_nodes_from_dict(G, d_node_clusterid)

genes_val = [1]*len(genes_rest)
d_genes_rest = dict(zip(genes_rest, genes_val))
colours_rest = color_nodes_from_dict_same(G, d_genes_rest, 'dimgrey')

colours_all = {**colours_rest, **colours_unsort}

colours_sorted = {key:colours_all[key] for key in G.nodes}
colours = list(colours_sorted.values())

In [ ]:
tsne_nodes = get_trace_nodes(posG_3D_complete_tsne, l_features, colours, 1)# size3d)
tsne_edges = get_trace_edges(G, posG_3D_complete_tsne, edge_colordark)

tsne_data = [tsne_edges, tsne_nodes]
#tsne_data = [tsne_nodes]

In [ ]:
print('Perplexity:', prplxty3d)
print('Early Exaggeration:', density3d)
print('Learning rate:', l_rate3d)
print('Iterations:', steps3d)

fig = pgo.Figure()
for i in tsne_data:
    fig.add_trace(i)
fig.update_layout(template='plotly_dark', showlegend=False, width=1200, height=1200,
                  scene=dict(
                      xaxis_title='',
                      yaxis_title='',
                      zaxis_title='',
                      xaxis=dict(nticks=0,tickfont=dict(
                            color='black')),
                      yaxis=dict(nticks=0,tickfont=dict(
                            color='black')),
                      zaxis=dict(nticks=0,tickfont=dict(
                            color='black')),
                    dragmode="turntable",
                    #annotations=annotations,
                ))
#py.iplot(fig)

plotly.offline.plot(fig, filename = 'output_plots/3Dportrait_tsne_'+feature+'_'+metric+'_'+organism+'.html', auto_open=True)

#### UMAP

In [ ]:
n_neighbors = 5 # balance between local and global structure in the data
spread = 0.9
min_dist = 0.1 # defines how dense points are stacked together 
metric='cosine'

In [ ]:
%%time 
umap_3D = embed_umap_3D(DM, n_neighbors, spread, min_dist, metric)

In [ ]:
# MAIN (if not all G.nodes)
posG_3Dumap = get_posG_3D(genes, umap_3D)

# REST (if genes = G.nodes then rest will be ignored / empty)

# get center of data points
xx=[]
yy=[]
zz=[]

for i in posG_3Dumap.values():
    xx.append(i[0])
    yy.append(i[1])
    zz.append(i[2]) 

cx = sum(xx)/len(genes)
cy = sum(yy)/len(genes)
cz = sum(zz)/len(genes)

# generate spherical coordinates for rest genes (without e.g. GO term or Disease Annotation)
indices = arange(0, len(genes_rest))
phi = arccos(1 - 2*indices/len(genes_rest))
theta = pi * (1 + 5**0.5) * indices

xm, ym, zm = max(posG_3Dumap.values())
r = math.sqrt((cx - xm)**2 + (cy - ym)**2 + (cz - zm)**2)
x, y, z = cx+r*cos(theta) * sin(phi),cy+r*sin(theta) * sin(phi), cz+r*cos(phi)

rest_points = []
for i,j,k in zip(x,y,z):
    rest_points.append((i,j,k))

posG_rest = dict(zip(genes_rest, rest_points))

posG_all = {**posG_3Dumap, **posG_rest}
posG_3D_complete_umap = {key:posG_all[key] for key in G.nodes()}

### UMAP CLUSTER Colors

In [ ]:
# SPECTRAL from sklearn
from sklearn.cluster import SpectralClustering

df_posG = pd.DataFrame(posG_3Dumap).T

n_clus = 50
color_method = 'spectral'

model = SpectralClustering(n_clusters=n_clus,n_components = 3, affinity='nearest_neighbors')
clusterid = model.fit_predict(df_posG)

d_node_clusterid = dict(zip(genes, clusterid))

# colour parameter from cluster 
colours_unsort = color_nodes_from_dict(G, d_node_clusterid)

genes_val = [1]*len(genes_rest)
d_genes_rest = dict(zip(genes_rest, genes_val))
colours_rest = color_nodes_from_dict_same(G, d_genes_rest, 'dimgrey')

colours_all = {**colours_rest, **colours_unsort}

colours_sorted = {key:colours_all[key] for key in G.nodes}
colours = list(colours_sorted.values())

In [ ]:
%%time 
umap_nodes = get_trace_nodes(posG_3D_complete_umap, l_features, colours, 1)#size3d)
umap_edges = get_trace_edges(G, posG_3D_complete_umap, edge_colordark) 

#umap_data = [umap_edges, umap_nodes]
umap_data = [umap_nodes]

In [ ]:
print('# Neighbours:', n_neighbors)
print('Spread:',spread)
print('Min. Distance:', min_dist)


fig = pgo.Figure()
for i in umap_data:
    fig.add_trace(i)

fig.update_layout(template='plotly_dark', showlegend=False, width=1200, height=1200,
                  scene=dict(
                      xaxis_title='',
                      yaxis_title='',
                      zaxis_title='',
                      xaxis=dict(nticks=0,tickfont=dict(
                            color='black')),
                      yaxis=dict(nticks=0,tickfont=dict(
                            color='black')),
                      zaxis=dict(nticks=0,tickfont=dict(
                            color='black')),    
                    dragmode="turntable",
                    #annotations=annotations,
                ))    

plotly.offline.plot(fig, filename = 'output_plots/3Dportrait_umap_'+feature+'_'+metric+'_'+organism+'.html', auto_open=True)

#### UMAP PORTRAIT EXPORT Coordinates

In [ ]:
# EXPORT as dataframe

df_3Dportrait_umap = pd.DataFrame(umap_posG3d).T
df_3Dportrait_umap.to_csv(r'output_csv/3DPortrait_umap_'+feature+'_'+organism+'_'+'lrate_'+l_rate3d+'steps_'+steps3d+'_'+metric+'.csv', index = True)

In [ ]:
# IMPORT and convert df to dict 
df_umap_posG3d = pd.read_csv('output_csv/3DPortrait_umap_'+feature+'_'+organism+'_'+'lrate_'+l_rate3d+'steps_'+steps3d+'_'+metric+'.csv', index_col=0)
X_umap = list(df_umap_posG3d['0'])
Y_umap = list(df_umap_posG3d['1'])
Z_umap = list(df_umap_posG3d['2'])

umap_posG3d = dict(zip(list(G.nodes()),zip(X_umap,Y_umap,Z_umap)))

___
## LANDSCAPES
___

### Define z-parameter
+ Choose one
___________

In [ ]:
# Degree Centrality

dict_z = {key:d_clos[key] for key in G.nodes()}
z_list = list(dict_z.values())

In [ ]:
# Essentiality

dict_z = d_essentiality_scores_sorted
z_list = list(dict_z.values())

In [ ]:
# Centrality Metrics average 

d_centr = {}
for i,j in df_centralities.iterrows():
    d_centr[i] = round((j[0]+j[1]+j[2]+j[3])/4,2)
    
d_binned = bin_nodes(d_centr)

d_node_bin = {}
for k,v in d_binned.items():
    for each in v:
        d_node_bin[each] = k
        
dict_z = {key:d_node_bin[key] for key in G.nodes()}

z_list = list(dict_z.values())

In [ ]:
# Function associated z-parameter

# feat_matrix is defined by the chosen Distance Matrix at the very beginning
features_counted = (feat_matrix == 1).astype(int).sum(axis=1)
d_features_counted = dict(zip(genes, list(features_counted)))
d_binned = bin_nodes(d_features_counted)

d_functions = {}
for k,v in d_binned.items():
    for each in v:
        d_functions[each] = k
        
d_rest = {}
for i in genes_rest: 
    d_rest[i] = 0
    
d_param_unsorted = {**d_functions, **d_rest}
dict_z = {key:d_param_unsorted[key] for key in G.nodes()}

z_list = list(dict_z.values())

________

### tSNE

In [ ]:
posG_land = {}
cc = 0
for k,v in posG_complete_tsne.items():
    posG_land[k] = (v[0],v[1],z_list[cc])
    cc+=1

### tSNE CLUSTER colors 

In [ ]:
# KMEANS
df_posG = pd.DataFrame(posG_tsne).T

color_method = 'kmeans'
n_clus = 50
n_iterations = 1000

iterations = range(1,n_iterations)
centers = []
clusterid = []
kmeans = KMeans(init='random', n_clusters=n_clus, random_state=0, max_iter=n_iterations, n_init=1, verbose=0)
ids = kmeans.fit_predict(df_posG) 
centers.append(kmeans.cluster_centers_)
clusterid.append(ids)

d_node_clusterid = get_node_clusterid(df_posG, n_clus, n_iterations)

# colour parameter from cluster 
colours_unsort = color_nodes_from_dict(G, d_node_clusterid)

genes_val = [1]*len(genes_rest)
d_genes_rest = dict(zip(genes_rest, genes_val))
colours_rest = color_nodes_from_dict_same(G, d_genes_rest, 'dimgrey')

colours_all = {**colours_rest, **colours_unsort}

colours_sorted = {key:colours_all[key] for key in G.nodes()}
colours = list(colours_sorted.values())

In [ ]:
tsne_nodes = get_trace_nodes(posG_land, l_features, colours, size3d)
tsne_edges = get_trace_edges(G, posG_land, edge_colordark)

tsne_data = [tsne_edges, tsne_nodes]
#tsne_data = [tsne_nodes]

In [ ]:
print('Perplexity:', prplxty)
print('Early Exaggeration:', density)
print('Learning rate:', l_rate)
print('Iterations:', steps)

fig = pgo.Figure()
for i in tsne_data:
    fig.add_trace(i)

fig.update_layout(template='plotly_dark', showlegend=False, width=1200, height=1200,
                  scene=dict(
                      xaxis_title='',
                      yaxis_title='',
                      zaxis_title='',
                      xaxis=dict(nticks=0,tickfont=dict(
                            color='black')),
                      yaxis=dict(nticks=0,tickfont=dict(
                            color='black')),
                      zaxis=dict(nticks=0,tickfont=dict(
                            color='black')),
                      aspectratio=dict(
                        x=1,
                        y=1,
                        z=1
                    ),
                    camera=dict(
                        center=dict(
                            x=0,
                            y=0,
                            z=0
                        ),
                        eye=dict(
                            x=1.96903462608,
                            y=-1.09022831971,
                            z=0.405345349304
                        ),
                        up=dict(
                            x=0,
                            y=0, 
                            z=1
                        )
                    ),
                    dragmode="turntable",
                    #annotations=annotations,
                ))
    

#py.iplot(fig)

plotly.offline.plot(fig, filename = 'output_plots/Landscapes_tsne_'+feature+'_'+metric+'_'+organism+'.html', auto_open=True)

### UMAP

In [329]:
posG_land = {}
cc = 0
for k,v in posG_complete_umap.items():
    posG_land[k] = (v[0],v[1],z_list[cc])
    cc+=1

### UMAP CLUSTER Colors

In [256]:
# KMEANS
df_posG = pd.DataFrame(posG_umap).T

color_method = 'kmeans'

n_clus = 30
n_iterations = 1000

iterations = range(1,n_iterations)
centers = []
clusterid = []
kmeans = KMeans(init='random', n_clusters=n_clus, random_state=0, max_iter=n_iterations, n_init=1, verbose=0)
ids = kmeans.fit_predict(df_posG) 
centers.append(kmeans.cluster_centers_)
clusterid.append(ids)

d_node_clusterid = get_node_clusterid(df_posG, n_clus, n_iterations)

# colour parameter from cluster 
colours_unsort = color_nodes_from_dict(G, d_node_clusterid)
genes_val = [1]*len(genes_rest)
d_genes_rest = dict(zip(genes_rest, genes_val))
colours_rest = color_nodes_from_dict_same(G, d_genes_rest, 'dimgrey')

colours_all = {**colours_rest, **colours_unsort}

colours_sorted = {key:colours_all[key] for key in G.nodes()}
colours = list(colours_sorted.values())

In [330]:
umap_nodes = get_trace_nodes(posG_land, l_features, colours, size3d)
umap_edges = get_trace_edges(G, posG_land, edge_colordark)

umap_data = [umap_edges, umap_nodes]
#umap_data = [umap_nodes]

In [331]:
print('# Neighbors: ', n_neighbors)
print('Spread: ', spread)
print('Min. Distance: ', min_dist)
print('Metric: ', metric)


fig = pgo.Figure()
for i in umap_data:
    fig.add_trace(i)

fig.update_layout(template='plotly_dark', showlegend=False, width=1200, height=1200,
                  scene=dict(
                      xaxis_title='',
                      yaxis_title='',
                      zaxis_title='',
                      xaxis=dict(nticks=0,tickfont=dict(
                            color='black')),
                      yaxis=dict(nticks=0,tickfont=dict(
                            color='black')),
                      zaxis=dict(nticks=0,tickfont=dict(
                            color='black')),
                      aspectratio=dict(
                        x=1,
                        y=1,
                        z=1
                    ),
                    camera=dict(
                        center=dict(
                            x=0,
                            y=0,
                            z=0
                        ),
                        eye=dict(
                            x=1.96903462608,
                            y=-1.09022831971,
                            z=0.405345349304
                        ),
                        up=dict(
                            x=0,
                            y=0,
                            z=1
                        )
                    ),
                    dragmode="turntable",
                    #annotations=annotations,
                ))


#py.iplot(fig)

plotly.offline.plot(fig, filename = 'output_plots/Landscapes_umap_'+feature+'_'+metric+'_'+organism+'.html', auto_open=True)

# Neighbors:  5
Spread:  0.9
Min. Distance:  0.1
Metric:  cosine


'output_plots/Landscapes_umap_rwr_cosine_Human.html'

### LANDSCAPE | EXPORT Coordinates x,y,z,rgba

In [ ]:
colours_r = []
colours_g = []
colours_b = []
colours_a = []
for i in colours:
    colours_r.append(i[0])
    colours_g.append(i[1])
    colours_b.append(i[2])
    colours_a.append(0.9)

In [ ]:
# t-SNE 

df_xyz_landscape_tsne = pd.DataFrame(posG_tsne, index=["x","y"]).T
df_xyz_landscape_tsne["z"] = z_list

df_xyz_landscape_tsne['r'] = colours_r
df_xyz_landscape_tsne['g'] = colours_g
df_xyz_landscape_tsne['b'] = colours_b
df_xyz_landscape_tsne['a'] = colours_a

df_xyz_landscape_tsne.to_csv(r'output_csv/landscape_tsne_XYZ_RGBA_'+feature+'_'+organism+'.csv', index = True)

In [ ]:
# UMAP 

df_xyz_landscape_umap = pd.DataFrame(posG_umap, index=["x","y"]).T
df_xyz_landscape_umap["z"] = z_list

df_xyz_landscape_umap['r'] = colours_r
df_xyz_landscape_umap['g'] = colours_g
df_xyz_landscape_umap['b'] = colours_b
df_xyz_landscape_umap['a'] = colours_a

df_xyz_landscape_umap.to_csv(r'output_csv/landscape_umap_XYZ_RGBA_'+feature+'_'+organism+'.csv', index = True)

## SPHERE

#### UMAP

### Define r-parameter
+ Choose one
___________

In [ ]:
# Essentiality
d_param = d_essentiality_scores_sorted

In [ ]:
# Closeness Centrality
d_clos_r = {}
for i,j in d_clos.items():
    d_clos_r[i] = round(j,2)
    
d_binned = bin_nodes(d_clos_r)

d_node_bin = {}
for k,v in d_binned.items():
    for each in v:
        d_node_bin[each] = k
        
d_param = {key:d_node_bin[key] for key in G.nodes()}

In [ ]:
# Degree Centrality
d_degs_r = {}
for i,j in d_deghubs.items():
    d_degs_r[i] = round(j,1)
    
d_binned = bin_nodes(d_degs_r)

d_node_bin = {}
for k,v in d_binned.items():
    for each in v:
        d_node_bin[each] = k
        
d_param = {key:d_node_bin[key] for key in G.nodes()}

In [168]:
# Centrality Metrics average 
d_centr = {}
for i,j in df_centralities.iterrows():
    d_centr[i] = round((j[0]+j[1]+j[2]+j[3])/4,2)
    
d_binned = bin_nodes(d_centr)

d_node_bin = {}
for k,v in d_binned.items():
    for each in v:
        d_node_bin[each] = k
        
d_param = {key:d_node_bin[key] for key in genes}  #G.nodes()}

In [481]:
# Function associated z-parameter

# feat_matrix is defined by the chosen Distance Matrix at the very beginning
features_counted = (feat_matrix == 1).astype(int).sum(axis=1)
d_features_counted = dict(zip(genes, list(features_counted)))
d_binned = bin_nodes(d_features_counted)

d_functions = {}
for k,v in d_binned.items():
    for each in v:
        d_functions[each] = k
        
d_rest = {}
for i in genes_rest: 
    d_rest[i] = 0
    
d_param_unsorted = {**d_functions, **d_rest}
d_param = {key:d_param_unsorted[key] for key in G.nodes()}

______

In [169]:
import umap 

def embed_umap_sphere(Matrix, n_neighbors, spread, min_dist, metric='cosine'):
    
    model = umap.UMAP(
        n_neighbors = n_neighbors, 
        spread = spread,
        min_dist = min_dist,
        metric = metric)

    sphere_mapper = model.fit(Matrix)

    return sphere_mapper

In [170]:
def get_posG_sphere(l_genes, sphere_mapper):
    x = np.sin(sphere_mapper.embedding_[:, 0]) * np.cos(sphere_mapper.embedding_[:, 1])
    y = np.sin(sphere_mapper.embedding_[:, 0]) * np.sin(sphere_mapper.embedding_[:, 1])
    z = np.cos(sphere_mapper.embedding_[:, 0])
    
    posG = {}
    cc = 0
    for entz in l_genes:
        posG[entz] = (x[cc],y[cc], z[cc])
        cc += 1
    
    return posG

In [171]:
n_neighbors = 4 # balance between local and global structure in the data
spread = 10.0
min_dist = 0.01 # defines how dense points are stacked together 
metric='cosine'

In [172]:
umap_sphere = embed_umap_sphere(DM, n_neighbors, spread, min_dist, metric)

In [173]:
posG_3Dsphere = get_posG_sphere(genes, umap_sphere)

In [174]:
# REST (if genes = G.nodes then rest will be ignored / empty)

# generate spherical coordinates for rest genes (without e.g. GO term or Disease Annotation)
indices = arange(0, len(genes_rest))
phi = arccos(1 - 2*indices/len(genes_rest))
theta = pi * (1 + 5**0.5) * indices

r = 0.5
x, y, z = r*cos(theta) * sin(phi), r*sin(theta) * sin(phi), r*cos(phi)

rest_points = []
for i,j,k in zip(x,y,z):
    rest_points.append((i,j,k))

posG_rest = dict(zip(genes_rest, rest_points))

posG_all = {**posG_3Dsphere, **posG_rest}
posG_complete_sphere = {key:posG_all[key] for key in G.nodes()}

### SPHERE Colors 

In [478]:
color_method = 'function'
d_to_be_coloured = d_functions # dict sorted by dict.values (that way the biggest value matches darkest colour of palette)

# Colouring
colour_groups = set(d_to_be_coloured.values())
colour_count = len(colour_groups)
pal = sns.color_palette('bright', colour_count)
palette = pal.as_hex()

d_colourgroups = {}
for n in colour_groups:
    d_colourgroups[n] = [k for k in d_to_be_coloured.keys() if d_to_be_coloured[k] == n]
    
d_colourgroups_sorted = {key:d_colourgroups[key] for key in sorted(d_colourgroups.keys())}

d_val_col = {}
for idx,val in enumerate(d_colourgroups_sorted):
    for ix,v in enumerate(palette):
        if idx == ix:
            d_val_col[val] = v
d_node_colour = {}
for y in d_to_be_coloured.items(): # y[0] = node id, y[1] = val
    for x in d_val_col.items(): # x[0] = val, x[1] = (col,col,col)
        if x[0] == y[1]:
            d_node_colour[y[0]]=x[1]  
            
d_rest_grey = {}
for i in genes_rest: 
    d_rest_grey[i] = 'dimgrey'
    
d_all_colours = {**d_node_colour, **d_rest_grey}
d_all_colours_sorted = {key:d_all_colours[key] for key in G.nodes()}

colours = list(d_all_colours_sorted.values())

In [483]:
%%time 
umap_sphere_withrad = get_posG_with_sphere_radius(G, posG_complete_sphere, d_param)

KeyboardInterrupt: 

In [479]:
sphere_nodes = get_trace_nodes(posG_complete_sphere, l_features, colours, 2) # size3d)
#sphere_edges = get_trace_edges(G, posG_complete_sphere, edge_colordark)

#tsne_data = [tsne_edges, tsne_nodes]
sphere_data = [sphere_nodes]

In [480]:
print('Metric:', metric)

fig = pgo.Figure()
for i in sphere_data:
    fig.add_trace(i)

fig.update_layout(template='plotly_dark', showlegend=False, autosize = True,
                  scene=dict(
                      xaxis_title='',
                      yaxis_title='',
                      zaxis_title='',
                      xaxis=dict(nticks=0,tickfont=dict(
                            color='black')),
                      yaxis=dict(nticks=0,tickfont=dict(
                            color='black')),
                      zaxis=dict(nticks=0,tickfont=dict(
                            color='black')),
                      #aspectratio=dict(
                      #  x=1,
                      #  y=1,
                      #  z=1
                    #),
                    dragmode="turntable",
                    #annotations=annotations,
                ))    
    
#py.iplot(fig)

plotly.offline.plot(fig, filename = 'output_plots/3Dsphere_umap_'+feature+'_'+metric+'_'+organism+'.html', auto_open=True)

Metric: cosine


'output_plots/3Dsphere_umap_disease_cosine_Human.html'

In [415]:
# WORK IN PROGRESS : print all other genes ( genes_rest) not in sphere
posG_rest = get_posG_2D(genes_rest, )

In [395]:
%%time
umap_sphere_trace_edges = get_trace_edges(G, umap_sphere_withrad, edge_colordark)
umap_sphere_trace_nodes = get_trace_umap_sphere(umap_sphere_withrad, l_features, colours, size3d)

# = [umap_sphere_trace_edges, umap_sphere_trace_nodes]
umap_sphere_data = [umap_sphere_trace_nodes]

NameError: name 'umap_sphere_withrad' is not defined

# WORK IN PROGRESS

+ COLOR SPECIFIC GENES (LIST)

In [ ]:
# ----------------------------------------------------        
# Essential genes 
# ----------------------------------------------------  
     
color_method = 'Essentiality' 
color = 'blue'

l_genes = essentials

# n o d e s 
colours = color_nodes_from_genelist(G, l_genes, color)

# e d g e s 
#d_edge_color_unsort = color_edges_from_genelist(G, l_genes, color)
#d_edge_color_sorted =  {key:d_edge_color_unsort[key] for key in G.edges()}
#edge_col = list(d_edge_color_sorted.values())

In [ ]:
def color_edges_from_genelist(G, l_genes, color):
    edge_lst = [(u,v) for u,v in G.edges(l_genes) if u in l_genes and v in l_genes]

    d_col_edges = {}
    for e in edge_lst:
        d_col_edges[e]=color
        
    d_nocol_edges = {}
    for e in G.edges():
        if e not in l_genes:
            d_nocol_edges[e]='lightgrey'
            
    d_both = {**d_col_edges, **d_nocol_edges}

    return d_both

In [ ]:
d_edge_color_unsort = color_edges_from_genelist(G, l_genes, color)

In [ ]:
# fix edge coloring 

In [ ]:
for i in d_edge_color_sorted.values():
    if i != 'lightgrey':
        print(i)

In [ ]:
# ----------------------------------------------------        
# Cancer Driver Genes
# ----------------------------------------------------  
     
color_method = 'Cancer Drivers' 
color = 'blue'

# n o d e s 
l_genes = l_cancerdrivers # my list of gene identifiers
colours = color_nodes_from_genelist(G, l_genes, color)

# e d g e s 
edge_color = list(color_edges_from_genelist(G, l_genes, color).values())

In [ ]:
# ----------------------------------------------------        
# Rare disease genes 
# ----------------------------------------------------  
     
color_method = 'Rare Disease Genes' 
color = 'red'

# l_genes = 

# n o d e s 
# colours = color_nodes_from_genelist(G, l_genes, 'red')

# e d g e s 
# edge_color = color_edges_from_genelist(G, l_genes, color)

## ANNOTATIONS 

In [ ]:
# ----------------------------------------------------        
# annotations 
# ----------------------------------------------------        

# Choose Specific Gene to annotate 
geneid = '4790'
genesyn = d_geneid_IBDsyn['4790'] 

x_gene = posG[geneid][0]
y_gene = posG[geneid][1]
z_gene = posG[geneid][2]

annotations = [dict ( 
                x=x_gene,
                y=y_gene,            
                z=z_gene, 
                showarrow=True,
                text = 'IBD GENE: '+genesyn,
                ay = -100,
                ax = -100,
                opacity = 0.8,
                arrowhead = 0,
                arrowwidth = 0.5 ,
                arrowcolor = "white" 
            )]

annotations = [dict(
                x=max(posG.values())[0],
                y=max(posG.values())[1],
                showarrow=True,
                text=disease_category,
                xanchor="right",
                ay=-100,
                ax=-100,
                opacity=0.8,
                arrowhead=0,
                arrowwidth=0.5,
                arrowcolor="white"
                )]

annotations3d = [dict(
                x=max(posG.values())[0],
                y=max(posG.values())[1],
                z=max(posG.values())[2],
                showarrow=True,
                text=disease_category,
                xanchor="right",
                ay=-100,
                ax=-100,
                opacity=0.8,
                arrowhead=0,
                arrowwidth=0.5,
                arrowcolor="white"
                )]

## Biological Node Properties

## ESSENTIALITY

In [ ]:
# Essentiality Score
essentials = []
for i in ess_id:
    essentials.append('entrez gene/locuslink:'+str(i))
    
nonessentials = []
for i in no_ess_id:
    nonessentials.append('entrez gene/locuslink:'+str(i))
    
major_score = 1
minor_score = -1

d_essentiality_scores = node_essentiality_scores(G, essentials, nonessentials, major_score, minor_score)
d_essentiality_scores_sorted = {key:d_essentiality_scores[key] for key in G.nodes()}

In [ ]:
# add statistical test for checking for essentiality to degree - correlation

## CANCER DRIVERS 

## RARE DISEASE GENES

+ CLUSTERS COLOURED

# ADDITIONAL STUFF

+ HUBS COLOURED

In [ ]:
feature = 'HUBS'

df_centralities = pd.read_csv('output_csv/Features_centralities_Dataframe_Human.csv', index_col=0)

d_deghubs = dict(zip(G.nodes(),df_centralities['degs']))
d_clos = dict(zip(G.nodes(), df_centralities['clos']))
d_betw = dict(zip(G.nodes(), df_centralities['betw']))
d_eigen = dict(zip(G.nodes(), df_centralities['eigen']))

cutoff = 3 # number of considered hub nodes 
d_hub_nodes = identify_hubs(d_deghubs, d_clos, d_betw, cutoff)

colours = color_majornodes(G, d_hub_nodes)
#colours = color_nodes_from_dict(G,d_hub_nodes)
edge_color = color_majornodes_outgoingedges(G, d_hub_nodes)

# Annotations 

d_hub_coords = {}
for node,hub in d_hub_nodes.items():
    for k,v in posG.items():
        if node == str(k):
            d_hub_coords[node] = v
            
hub_x = []
hub_y = []
hub_z = []
for node,coords in d_hub_coords.items():
    hub_x.append(coords[0])
    hub_y.append(coords[1])
    hub_z.append(coords[2])
    
annotations = [dict(
                x=hub_x,
                y=hub_y,
                z=hub_z,
                showarrow=True,
                text="Hubs: "+str(i),
                xanchor="right",
                ay=-100,
                ax=-100,
                opacity=0.8,
                arrowhead=0,
                arrowwidth=0.5,
                arrowcolor="white"
                ) for i, hub_x, hub_y, hub_z in zip(range(1,len(d_hub_nodes)+1),hub_x, hub_y, hub_z)]

+ CLOSENESS CENTRALITY COLOR 

In [ ]:
feature = 'CLOSE'

# Define colour parameter
d_to_be_coloured = d_clos # dict sorted by dict.values (that way the biggest value matches darkest colour of palette)


# Colouring
colour_groups = set(d_to_be_coloured.values())
colour_count = len(colour_groups)
palette = sns.color_palette('Reds', colour_count)

d_colourgroups = {}
for n in colour_groups:
    d_colourgroups[n] = [k for k in d_to_be_coloured.keys() if d_to_be_coloured[k] == n]
    
d_colourgroups_sorted = {key:d_colourgroups[key] for key in sorted(d_colourgroups.keys())}

d_val_col = {}
for idx,val in enumerate(d_colourgroups_sorted):
    for ix,v in enumerate(palette):
        if idx == ix:
            d_val_col[val] = v
d_node_colour = {}
for y in d_to_be_coloured.items(): # y[0] = node id, y[1] = val
    for x in d_val_col.items(): # x[0] = val, x[1] = (col,col,col)
        if x[0] == y[1]:
            d_node_colour[y[0]]=x[1]
            
# SORT dict based on G.nodes
d_node_colour_sorted = dict([(key, d_node_colour[key]) for key in G.nodes()])

l_col_clos = list(d_node_colour_sorted.values())

colours = l_col_clos 

### COLOURING SUBCATEGORIES e.g. Disease subcategory (i.e. functional property)

In [ ]:
# input = dict

n = len(sub_categories)
colors = generate_colorlist_nodes(n)

doid_coloured = {}
c=0
for k,v in sub_categories.items():
    doid_coloured[k[1]]=colors[c]
    c+=1
    
d_gene_colours = {}
for doid, gene in d_do_genes.items(): 
    for i in gene:
        for do, col in doid_coloured.items():
            if doid == do:
                d_gene_colours[i]=col

# SORT dict based on G.nodes
d_gene_colours_sorted = {key:d_gene_colours[key] for key in G.nodes()}

l_col_subcat = list(d_gene_colours_sorted.values())


# NODE COLOURING based on Subcategory
colours = l_col_subcat 
edge_color = 'lightgrey'

#sns.palplot(colours)

### COLOURING DISEASE SCORES

In [ ]:
# TO DO : MAKE FUNCTION FOR COLORING 

gene_list = gene_list_retisarc



# COLOURS GENERATED / disease score and neighboring nodes

n = len(gene_list)
color_major = generate_colorlist_nodes(n)
 
factor = 1.7 # the higher the lighter
color_minor = []
for i in color_major:
    r,g,b = hex_to_rgb(i)
    color_light = adjust_color_lightness(r,g,b,factor)
    color_minor.append(color_light)
    
    
# ------------------------------------------------------
# NODES coloured 

# Major nodes coloured
d_col_major = {}
for n in d_major_score.keys():
    for i in color_major:
        d_col_major[n] = i

d_col_minor = {}
for node,col in d_col_major.items():
    for node in d_col_major.keys():
        for nd,neigh in major_neigh.items():
            for n in neigh:
                for i in color_minor:
                    if node==nd and n not in d_col_major.keys():
                        d_col_minor[n] = i
                    
d_col = {**d_col_major,**d_col_minor}

d_grey = {}
for i in G.nodes():
    if i not in d_col.keys():
        d_grey[i] = 'lightgrey'
        
        
d_col_all = {**d_col_major, **d_col_minor, **d_grey}
d_col_all_sorted = {key:d_col_all[key] for key in G.nodes()}

l_col_all = list(d_col_all_sorted.values())

colours = l_col_all

# ------------------------------------------------------
# EDGES coloured 

edge_lst = []
for edge in G.edges():
    for e in edge:
        for node in d_col_major.keys():
            if e == node:
                edge_lst.append(edge)
                
d_col_edges = {}
for e in edge_lst:
    for node,col in d_col_major.items():
        if e[0] == node:
            d_col_edges[e]=col
        elif e[1] == node:
            d_col_edges[e]=col

d_grey_edges = {}
for edge in G.edges():
    if edge not in d_col_edges.keys(): 
        d_grey_edges[edge] = 'lightgrey'
        
d_edges_all = {**d_col_edges, **d_grey_edges}

# Sort according to G.edges()
d_edges_all_sorted = {key:d_edges_all[key] for key in G.edges()}

edge_color = list(d_edges_all_sorted.values())